The goal of this notebook is to implement multivariate linear regression from scratch using gradient descent and validate it.
The algorithm is validated on the glucose dataset discussed in the sklearn repository.

In [177]:
import pandas as pd
import numpy as np

from sklearn import datasets

def normalize(X):
    X_opt = (X-np.mean(X, axis=0)) / np.std(X, axis=0)
    return X_opt

diabetes = datasets.load_diabetes()

df = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)

X = diabetes.data

X = normalize(X) 


y = diabetes.target.reshape(-1, 1) # Converting to Column vector   
# Inserting bias feature (x0 = 1) into features for all examples
X = np.concatenate((np.ones((X.shape[0],1)), X), axis=1)

### Creating a test train split

In [178]:
idx = np.random.permutation(range(len(y)))
ratio_for_split = 0.8
split_index = int(X.shape[0]*ratio_for_split)

## training set
X_train, y_train = X[idx[:split_index]], y[idx[:split_index]]

## test set
X_test, y_test = X[idx[split_index:]], y[idx[split_index:]]

In [179]:
class MultivariateLinearRegression:
    def __init__(self, dimensions, alpha, epochs):
        # intialize weights
        self.w = np.zeros((dimensions, 1))
        self.alpha = alpha # Learning rate
        self.epochs = epochs # Iterations/epochs T
        # Cost function values during the training
        self.costs_train = []
    
    def fit(self, X, y):
        for epoch in np.arange(1, self.epochs+1):
            
            # Predictions
            preds = self.predict(X)
            
            # Costs
            cost_value = self.cost(preds, y)        

            # Storing the costs
            self.costs_train.append(cost_value)
            
            # Finding the Derivatives
            dw = (1/X.shape[0]) * np.matmul(X.T , (preds - y))
            # Updating parameters
            self.update(dw)    
        return preds
            
    def update(self, dw):
        self.w -= self.alpha * dw
        
    # Model prediction
    def predict(self, X):
        preds = np.matmul(X, self.w)
        return preds
    
    def get_parameters(self):
        return self.w
    
    def cost(self, preds, y):
        Cost = (1/2) * np.mean( np.power( np.sum(preds-y), 2 ) )
        return Cost

### Setting the `dimensions`, learning rate `alpha` and the `epochs`.

Note: Using the complete training set

In [180]:
model = MultivariateLinearRegression(X.shape[1], 0.06, 80)

In [181]:
predictions = model.fit(X, y)
updated_parameters = model.get_parameters()

In [182]:
# sanity check
predictions.shape

(442, 1)

In [183]:
y.shape

(442, 1)

In [184]:
MSE = np.mean((y-predictions)**2)
RSS = MSE/(np.std(y)**2)
print("MSE is ", MSE)

Rsq = 1 - RSS
print("R^2 is ", Rsq)

MSE is  2882.453840763284
R^2 is  0.5139106591655576


### Using the test train split

In [185]:
model_with_split = MultivariateLinearRegression(X_train.shape[1], 0.06, 100)

### Verifying with the `model_with_split` on the test data

In [186]:
predictions_with_split = model_with_split.fit(X_test, y_test)
updated_parameters_with_split = model_with_split.get_parameters()

In [187]:
MSE_with_split = np.mean((y_test-predictions_with_split)**2)
RSS_with_split = MSE_with_split/(np.std(y_test)**2)
print("MSE is ", MSE_with_split)

Rsq_with_split = 1 - RSS_with_split
print("R^2 is ", Rsq_with_split)

MSE is  3460.5842541494626
R^2 is  0.4869385698893116


## Scikit Learn Validation of MSE Score

In [188]:
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import LinearRegression

In [189]:
reg = LinearRegression()
reg.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [190]:
print("MSE: ", mean_squared_error(y, predictions))
print("R2 score: ", r2_score(y, predictions))

MSE:  2882.453840763284
R2 score:  0.5139106591655576
